In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

In [8]:
dtype = {'MonthlyCharges': 'float'}
df = pd.read_csv('../data/telco_churn_raw.csv', dtype=dtype)

There are 11 rows which are missing the "TotalCharges" value. All of them have less than 1 month tenure so they are likely just new customers who haven't paid their first bill yet. Just fill in as 0. 

In [9]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0.0)
df['TotalCharges'] = df['TotalCharges'].astype(float)

Since I want this first set for pretty EDA I'll change SeniorCitizen to 'yes' and 'no'

In [10]:
df['SeniorCitizen'] = df['SeniorCitizen'].replace(0, 'No')
df['SeniorCitizen'] = df['SeniorCitizen'].replace(1, 'Yes')

In [11]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [12]:
# # Comment this out when not running
# df.to_csv('../data/telco_churn_clean.csv', index=False)

### Make Numeric
The "clean" dataset is for the EDA (Pretty labels and what-not) so this next dataset is for training models.

In [48]:
df = pd.read_csv('../data/telco_churn_clean.csv')

In [49]:
# gender 
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['gender'].value_counts()

### consolidate into binary columns
# Multiple Lines
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

# Internet Service
df['InternetService'] = df['InternetService'].replace('Fiber optic', 'Yes')
df['InternetService'] = df['InternetService'].replace('DSL', 'Yes')

# Online Security
df['OnlineSecurity'] = df['OnlineSecurity'].replace('No internet service', 'No')

# Online Backup
df['OnlineBackup'] = df['OnlineBackup'].replace('No internet service')

# Device Protection
df['DeviceProtection'] = df['DeviceProtection'].replace('No internet service')

# Tech Support
df['TechSupport'] = df['TechSupport'].replace('No internet service', 'No')

# Streaming TV
df['StreamingTV'] = df['StreamingTV'].replace('No internet service', 'No')

# Streaming Movies
df['StreamingMovies'] = df['StreamingMovies'].replace('No internet service', 'No')

# Contract
df['Contract_MtM'] = df['Contract'].apply(lambda x: 1 if x=='Month-to-month' else 0)
df['Contract_1yr'] = df['Contract'].apply(lambda x: 1 if x=='One year' else 0)
df['Contract_2yr'] = df['Contract'].apply(lambda x: 1 if x=='Two year' else 0)
df.drop('Contract', axis=1, inplace=True)

# Payment Method
df['Payment_Electronic'] = df['PaymentMethod'].apply(lambda x: 1 if x=='Electronic check' else 0)
df['Payment_Mail'] = df['PaymentMethod'].apply(lambda x: 1 if x=='Mailed check' else 0)
df['Payment_Bank'] = df['PaymentMethod'].apply(lambda x: 1 if x=='Bank transfer (automatic)' else 0)
df['Payment_Credit'] = df['PaymentMethod'].apply(lambda x: 1 if x=='Credit card (automatic)' else 0)
df.drop('PaymentMethod', axis=1, inplace=True)

In [54]:
### turn binary into numeric
bin_cols = df.nunique()[df.nunique() == 2].index
for col in bin_cols: 
    if df[col].dtype != int:
        df[col] = df[col].replace('Yes', 1)
        df[col] = df[col].replace('No', 0)

In [56]:
df.dtypes

customerID             object
gender                  int64
SeniorCitizen           int64
Partner                 int64
Dependents              int64
tenure                  int64
PhoneService            int64
MultipleLines           int64
InternetService         int64
OnlineSecurity          int64
OnlineBackup            int64
DeviceProtection        int64
TechSupport             int64
StreamingTV             int64
StreamingMovies         int64
PaperlessBilling        int64
MonthlyCharges        float64
TotalCharges          float64
Churn                   int64
Contract_MtM            int64
Contract_1yr            int64
Contract_2yr            int64
Payment_Electronic      int64
Payment_Mail            int64
Payment_Bank            int64
Payment_Credit          int64
dtype: object

All of the columns are numeric. Ready for training!

In [57]:
# # Comment this out when not running
# df.to_csv('../data/telco_churn_numeric.csv', index=False)